In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
cd ~/code/snc

In [ ]:
# importing tree_gsm with implemented capacity constrains extension (branch egor/gsm_capacity_constraints)
import snc.meio.gsm.tree_gsm as tree_gsm

In [ ]:
# importing numerical simulator
from snc.experiment.numerical_simulator import simulate

In [ ]:
# importing function which will generate serial network as in 
# section 4. Numerical Experiments from Graves and Schoenmeyr 2016
from snc.experiment.basic_serial_network import create_serial_stages

In [ ]:
# generate history of demand using the parameters in Section 4

n=100000 # length of simulation (number of days)
loc = 40 # daily demand mean
scale = 20 # daily demand std

np.random.seed(seed=8675309)
demand_history = np.maximum(np.round(np.random.normal(loc,scale,size=n)),0)

In [ ]:
# Here we will try to replicate scenarios in Table 5

added_cost_prof = "constant"
lead_time_prof = "upstream_heavy"

stages = create_serial_stages(added_cost_prof=added_cost_prof,lead_time_prof=lead_time_prof)

In [ ]:
# first without capacity constraints

gsm = tree_gsm.GuaranteedServiceModelTree(stages,propagate_bounds=True)
solution = gsm.find_optimal_solution(root_stage_id="2")

In [ ]:
# verify safety stock levels are as in the table
safety_stocks = tree_gsm.compute_expected_inventories(solution.policy,stages)
safety_stocks

In [ ]:
# simulate

capacity_constraints = {}
indep_inv_histories = simulate(stages,
                               solution.policy,
                               solution.base_stocks,
                               capacity_constraints,
                               demand_history,
                               stockout_stages=[])

In [ ]:
# verify service levels (in this serial example k = 2, which which corresponds to 97% service level)

np.mean(indep_inv_histories["1"] >= 0),np.mean(indep_inv_histories["5"] >= 0)

In [ ]:
# now lets place capacity constraint at stage 1

cap_loc = "1"
stages = create_serial_stages(added_cost_prof=added_cost_prof,lead_time_prof=lead_time_prof)
stages[cap_loc].cap_constraint = 45

In [ ]:
# rerun gsm optimization and verify safety stock levels from the table
# note that expected backlog is not computed here but it will result in safety stocks being less by 30 units
# regardless of the capacity constraint location (see text just to the right of table 5)
# hence in the safety stock levels computed below level is higher by exactly 30

gsm = tree_gsm.GuaranteedServiceModelTree(stages,propagate_bounds=True)
solution = gsm.find_optimal_solution(root_stage_id="2")

safety_stocks = tree_gsm.compute_expected_inventories(solution.policy,stages)
safety_stocks

In [ ]:
# simulate

capacity_constraints = {cap_loc:45}
indep_inv_histories = simulate(stages,
                               solution.policy,
                               solution.base_stocks,
                               capacity_constraints,
                               demand_history,
                               stockout_stages=[])

In [ ]:
# verify expirical average inventory levels with the values from the table

np.mean(indep_inv_histories["1"]),np.mean(indep_inv_histories["3"])

In [ ]:
# now check the service levels

np.mean(indep_inv_histories["1"] >= 0),np.mean(indep_inv_histories["3"] >= 0)

As can be seen, simulating the network with the prescribed base stocks from implemented GSM with capacity constraints results in much lower service level at demand stage 1 (expected 97%)

There can be several sources of discrepancy:
* numerical simulator does not execute replenishment policy correctly
* analytical solution underestimates the variance of inventory under censored reorder policy

As can be seen from the empirical inventory averages matching the expected values it is unlikely that numerical simulator is the cause. But it is still very worth checking that re-order dynamics are simulated exactly as assumed by the authors of the paper

One the other hand, we have a suspicion that the formula used for computing basestocks (equation 9) does not take into account stochasticity of replenishment orders but rather uses deterministic upper bound derived from capacity constraint. As a result the gap between cumulative demand and cumulative replenishments at any point of time is underestimated 